In [1]:
import json
import pandas as pd
from typing import Dict, List, Optional, Union
from pydantic import BaseModel, Field
from datasets import Dataset
from bespokelabs import curator
import argparse
from pathlib import Path
from typing import List, Dict

import os
os.environ["CURATOR_VIEWER"] = "1"

/Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ============================================================================
# Pydantic Models for Structured Outputs
# ============================================================================

class MolecularProperty(BaseModel):
    """Molecular property prediction with confidence and reasoning."""
    property_name: str = Field(description="Name of the molecular property")
    predicted_value: float = Field(description="Predicted numerical value")
    confidence: str = Field(description="Confidence level: high, medium, low")
    reasoning: str = Field(description="Chemical reasoning for the prediction")
    key_structural_features: List[str] = Field(description="Important structural features affecting the property")


In [3]:
# ============================================================================
# Core Dataset Generators
# ============================================================================
class PropertyPredictor(curator.LLM):
    """Generates property prediction tasks with reasoning."""
    
    response_format = MolecularProperty
    
    def prompt(self, input: Dict) -> str:
        smiles = input['canonical_smiles']
        property_type = input.get('standard_type', 'LogD')

        return f"""You are an expert medicinal chemistry research assistant. You have knowledge in areas including:

- Drug design and development - structure-activity relationships (SAR), lead optimization, pharmacophore modeling
- Synthetic chemistry - reaction mechanisms, synthetic routes, protecting group strategies
- Pharmacology - drug-target interactions, pharmacokinetics (ADME), pharmacodynamics
- Computational chemistry - molecular modeling, QSAR and ADME predictions
- Analytical techniques - NMR, MS, HPLC and other characterization methods
- Medicinal chemistry strategies - bioisosteres, prodrugs, fragment-based drug design

Analyze the following molecule and predict its {property_type}.

Molecule (SMILES): {smiles}

Provide a detailed prediction including:
1. The predicted {property_type} value
2. Your confidence level
3. Chemical reasoning based on structural features
4. Key structural features that influence this property

Consider factors like:
- Lipophilicity and hydrophilicity
- Molecular weight and size
- Functional groups present
- Aromatic systems
- Hydrogen bonding potential
- Charge distribution"""

    def parse(self, input: Dict, response: MolecularProperty) -> Dict:
        return {
            "smiles": input['canonical_smiles'],
            "actual_value": input.get('standard_value'),
            "predicted_value": response.predicted_value,
            "property_type": input.get('standard_type', 'LogD'),
            "confidence": response.confidence,
            "reasoning": response.reasoning,
            "key_features": response.key_structural_features,
        }

In [4]:
df = pd.read_csv('/Users/ldodda/Documents/Codes/GraphGen/test_logd.csv')

In [ ]:
df.sample(2000)

,smiles,logd
4935,O=CNc1cc([C@@H](O)CNCCc2ccc(Oc3ccccc3)cc2)ccc1O,1.95
8530,COc1cccc([C@@H](Oc2ccc3c(cnn3-c3ccc(F)cc3)c2)[...,6.00
5737,CN(C)c1nc(=O)n(-c2ccccc2)c2cc(Cl)ccc12,2.00
146,N=C(Nc1ccc2c(c1)CCN2C1CCNCC1)c1cccs1,-1.78
9757,C[C@@H]1CN(c2noc(C3(C)COC3)n2)CCN1c1ncc(OCc2cc...,3.00
1008,CCCCOCCOc1nc(-c2ccc(C(=O)O)c(F)c2)sc1C,1.40
1235,FC(F)(F)c1cccc2c1nnn2CC1CC1,3.08
5660,Cc1ccc2c(=O)[nH]c(-c3ccc(C4(O)COC4)cc3)cn12,1.70
4072,c1ccc2c(c1)nc(CCc1ccc3c(c1)OCO3)c1cccn12,3.84
4460,CO[C@]1(C)C[C@H](O[C@H]2[C@H](C)[C@@H](O[C@@H]...,4.40


In [6]:
df.columns = ['canonical_smiles', 'standard_value']
df['standard_type'] = 'LogD'

In [7]:
mol_data_dict = df.sample(20).to_dict(orient='records')

In [8]:
logd_dataset = Dataset.from_list(mol_data_dict)

In [9]:
logd_dataset

Dataset({
    features: ['canonical_smiles', 'standard_value', 'standard_type'],
    num_rows: 20
})

In [10]:
predictor = PropertyPredictor(
        model_name="bedrock/us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        backend="litellm"
)

[11/02/25 20:46:04] INFO     Getting rate limits for model:                 ]8;id=939428;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/litellm_online_request_processor.py\litellm_online_request_processor.py]8;;\:]8;id=551347;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/litellm_online_request_processor.py#213\213]8;;\
                             bedrock/us.anthropic.claude-3-7-sonnet-2025021                                        
                             9-v1:0                                                                                

[11/02/25 20:46:06] INFO     Test call headers: {}                          ]8;id=981016;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/litellm_online_request_processor.py\litellm_online_request_processor.py]8;;\:]8;id=164215;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/litellm_online_request_processor.py#199\199]8;;\

In [11]:
print("Generating predictions...")
results = predictor(logd_dataset)


Generating predictions...


[11/02/25 20:46:07] INFO     Running LiteLLMOnlineRequestProcessor completions with   ]8;id=128095;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=919924;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py#133\133]8;;\
                             model:                                                                                
                             bedrock/us.anthropic.claude-3-7-sonnet-20250219-v1:0                                  

                    INFO     Preparing request file(s) in                             ]8;id=864453;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=231522;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py#233\233]8;;\
                             /Users/ldodda/.cache/curator/48c666ec4ccfa49b                                         

                    INFO     Wrote 20 requests to                                     ]8;id=49159;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=63811;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py#315\315]8;;\
                             /Users/ldodda/.cache/curator/48c666ec4ccfa49b/requests_0                              
                             .jsonl.                                                                               

                    WARNING  No manual max_requests_per_minute set, and        ]8;id=867071;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py\base_online_request_processor.py]8;;\:]8;id=996485;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py#197\197]8;;\
                             headers based detection failed, using default                                         
                             value of 200                                                                          

...

                    INFO     Status tracker: Tasks - Started: 20, In Progress: ]8;id=441849;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py\base_online_request_processor.py]8;;\:]8;id=743967;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py#460\460]8;;\
                             0, Succeeded: 20, Failed: 0, Already Completed: 0                                     
                             Errors - API: 0, Rate Limit: 0, Other: 0, Total:                                      
                             0                                                                                     

                    INFO     Read 20 responses.                                       ]8;id=224016;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=30541;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py#444\444]8;;\

                    INFO     Finalizing writer                                        ]8;id=827346;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=835493;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py#453\453]8;;\

                    INFO     Creating a file with all failed requests                 ]8;id=439691;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=774892;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py#462\462]8;;\

                    INFO     Created file with failed requests at                     ]8;id=242492;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=761372;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py#490\490]8;;\
                             /Users/ldodda/.cache/curator/48c666ec4ccfa49b/failed_req                              
                             uests.jsonl                                                                           

In [12]:
print("\n📊 Results:")
logD_dataset_df = results.dataset.to_pandas()


📊 Results:


In [13]:
logD_dataset_df

,smiles,actual_value,predicted_value,property_type,confidence,reasoning,key_features
0,N#CCc1ccccc1C#Cc1ccc(CCC(=O)O)c(F)c1,1.35,2.8,LogD,medium,The molecule contains both lipophilic and hydr...,"[Carboxylic acid group, Nitrile group, Two aro..."
1,Cc1ccc(-c2nnc(N)s2)cc1C,2.95,2.3,LogD,medium,The molecule contains a dimethylated phenyl ri...,"[Dimethylated phenyl ring, Thiazole ring, Prim..."
2,CNC(=O)c1nn(C)c2ccc(Nc3nccc(-c4cc(CN(C)C)n(C)n...,2.20,2.1,LogD,medium,The molecule contains both lipophilic and hydr...,"[Pyrazole ring system, Pyrimidine ring, Dimeth..."
3,Cc1noc(C)c1-c1cccc(COc2ccc3c(c2)OCC3CC(=O)O)c1,1.57,2.3,LogD,medium,The molecule contains multiple structural comp...,"[Carboxylic acid group, Isoxazole ring, Benzod..."
4,Cn1c(=O)c2cc(C(=O)NCCCN3CCCCCC3)sc2c2ccccc21,1.89,2.3,LogD,medium,This molecule contains competing lipophilic an...,"[Fused thiazole-benzene-benzothiazole system, ..."
5,CCc1cc(-c2ncnc3[nH]ccc23)ccc1CNC(=O)c1ccc(C(C)...,2.35,3.5,LogD,medium,The molecule contains multiple lipophilic comp...,"[Aromatic rings (3 total), Secondary amide, In..."
6,Cc1ccccc1-c1cccc(COc2ccc(S(=O)(=O)CC(=O)O)cc2)c1,2.35,2.4,LogD,high,The molecule Cc1ccccc1-c1cccc(COc2ccc(S(=O)(=O...,"[Biphenyl core structure (lipophilic), Sulfone..."
7,CCOc1cc(CN2CCC(NC(=O)c3cncc(C)c3)CC2)cc(OCC)c1Cl,3.43,2.8,LogD,medium,The molecule CCOc1cc(CN2CCC(NC(=O)c3cncc(C)c3)...,"[Dichlorinated, diethoxy-substituted aromatic ..."
8,CNCC(=O)N[C@H](c1cccc(F)c1N1CCN(C(=O)[C@H](C)C...,3.10,2.8,LogD,medium,The molecule contains a mix of lipophilic and ...,[Aromatic rings (fluorophenyl and chlorophenyl...
9,O=C(NC[C@@H](O)CN1CCC(Oc2ccc(Cl)c(Cl)c2)CC1)c1...,3.70,2.5,LogD,medium,The molecule contains a mix of lipophilic and ...,"[Dichlorophenyl group (lipophilic), Piperidine..."


In [14]:
results.dataset

Dataset({
    features: ['smiles', 'actual_value', 'predicted_value', 'property_type', 'confidence', 'reasoning', 'key_features'],
    num_rows: 20
})

In [15]:
results.dataset.to_list()

[{'smiles': 'N#CCc1ccccc1C#Cc1ccc(CCC(=O)O)c(F)c1',
  'actual_value': 1.35,
  'predicted_value': 2.8,
  'property_type': 'LogD',
  'confidence': 'medium',
  'reasoning': "The molecule contains both lipophilic and hydrophilic components that affect its LogD value. The lipophilic elements include two aromatic rings, an alkyne linker, and several aliphatic chains (the CCN#C, CCC(=O)O), which increase the LogD. The fluorine substituent also contributes to lipophilicity. The hydrophilic elements include the carboxylic acid group and the nitrile group, which decrease the LogD through their ability to form hydrogen bonds and their polarity. At physiological pH (7.4), the carboxylic acid (pKa ~4-5) would be predominantly in the ionized state, significantly decreasing the LogD compared to the neutral molecule's LogP. The nitrile group, while not ionizable, contributes moderate polarity. Overall, the balance of these structural features suggests a moderately lipophilic compound with a LogD aroun

In [16]:
output_file = "logd_predictor.jsonl"
with open(output_file, 'w') as f:
    for item in results.dataset.to_list():
        f.write(json.dumps(item) + '\n')

In [17]:
"""
Data Preparation Script for Llama 3.2 3B Fine-tuning
Converts bespokelabs-curator dataset to instruction-following format
"""

system_prompt = """You are an expert medicinal chemistry research assistant. You have knowledge in areas including:

- Drug design and development - structure-activity relationships (SAR), lead optimization, pharmacophore modeling
- Synthetic chemistry - reaction mechanisms, synthetic routes, protecting group strategies
- Pharmacology - drug-target interactions, pharmacokinetics (ADME), pharmacodynamics
- Computational chemistry - molecular modeling, QSAR and predicting ADME properties
- Analytical techniques - NMR, MS, HPLC and other characterization methods
- Medicinal chemistry strategies - bioisosteres, prodrugs, fragment-based drug design

You analyze molecular structures using SMILES notation and provide accurate predictions for LogD values (distribution coefficient at pH 7.4). 
Your predictions are based on careful analysis of structural features, lipophilic and hydrophilic contributions, and established cheminformatics principles.
"""
class DatasetConverter:
    """Convert molecular property dataset to instruction-following format."""
    
    def __init__(self, input_path: str, output_path: str):
        self.input_path = Path(input_path)
        self.output_path = Path(output_path)
        
        if not self.input_path.exists():
            raise FileNotFoundError(f"Input file not found: {input_path}")
    
    def convert_to_instruction_format(self) -> None:
        """Convert dataset to Llama 3.2 instruction format."""
        
        print(f"Reading dataset from: {self.input_path}")
        
        with open(self.input_path, 'r') as f_in:
            with open(self.output_path, 'w') as f_out:
                example_count = 0
                
                for line_num, line in enumerate(f_in, 1):
                    try:
                        example = json.loads(line)
                        converted = self._create_instruction_example(example)
                        f_out.write(json.dumps(converted) + '\n')
                        example_count += 1
                        
                    except json.JSONDecodeError as e:
                        print(f"Warning: Invalid JSON on line {line_num}: {e}")
                    except KeyError as e:
                        print(f"Warning: Missing field on line {line_num}: {e}")
        
        print(f"✓ Converted {example_count} examples")
        print(f"✓ Output saved to: {self.output_path}")
    
    @staticmethod
    def _create_instruction_example(entry: Dict) -> Dict:
        """
        Convert single entry to instruction format.
        
        Args:
            entry: Original entry with SMILES, values, and reasoning
            
        Returns:
            Dictionary with 'text' field containing formatted conversation
        """
        
        smiles = entry['smiles']
        actual_value = entry['actual_value']
        reasoning = entry['reasoning']
        key_features = ', '.join(entry['key_features'])
        
        # Create the full conversation in Llama 3.2 format
        text = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_prompt}
<|eot_id|><|start_header_id|>user<|end_header_id|>
Analyze the following molecule and predict its LogD value.

SMILES: {smiles}

Key Structural Features: {key_features}

Chemical Analysis and Reasoning:
{reasoning}

Based on this analysis, what is the predicted LogD value for this molecule?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Based on the detailed structural analysis, considering the balance of lipophilic and hydrophilic features:

The predicted LogD value is: **{actual_value}**

This prediction reflects:
- The lipophilic contributions from the {key_features.split(',')[0] if ',' in key_features else 'aromatic and aliphatic components'}
- The hydrophilic contributions from polar functional groups
- The overall three-dimensional distribution of the molecular features
- The resulting membrane permeability and aqueous solubility balance
<|eot_id|>"""
        
        return {"text": text}
    
    def validate_output(self) -> bool:
        """Validate the converted dataset."""
        
        print("\nValidating dataset...")
        
        try:
            examples = []
            with open(self.output_path, 'r') as f:
                for i, line in enumerate(f):
                    ex = json.loads(line)
                    if 'text' not in ex:
                        raise ValueError(f"Line {i}: missing 'text' field")
                    examples.append(ex)
            
            print(f"✓ Valid JSONL format")
            print(f"✓ Number of examples: {len(examples)}")
            
            # Show statistics
            text_lengths = [len(ex['text']) for ex in examples]
            print(f"✓ Average text length: {sum(text_lengths) / len(text_lengths):.0f} chars")
            print(f"✓ Min length: {min(text_lengths)} chars")
            print(f"✓ Max length: {max(text_lengths)} chars")
            
            # Show sample
            print(f"\n--- Sample Example (first 500 chars) ---")
            print(examples[0]['text'][:500] + "...\n")
            
            return True
            
        except Exception as e:
            print(f"✗ Validation failed: {e}")
            return False

In [18]:
converter = DatasetConverter('./logd_predictor.jsonl', './logd_formatted.jsonl')
converter.convert_to_instruction_format()

Reading dataset from: logd_predictor.jsonl
✓ Converted 20 examples
✓ Output saved to: logd_formatted.jsonl


In [19]:
!pip install transformers peft bitsandbytes accelerate datasets trl wandb


In [20]:
from datasets import Dataset


def load_jsonl_data(file_path):
    """Load JSONL dataset."""
    texts = []
    with open(file_path, 'r') as f:
        for line in f:
            example = json.loads(line)
            texts.append(example['text'])
    return texts

DATASET_PATH = './logd_formatted.jsonl'
print(f"Loading dataset from {DATASET_PATH}...")
texts = load_jsonl_data(DATASET_PATH)
print(f"Loaded {len(texts)} examples")

# Create HF dataset
dataset = Dataset.from_dict({
    'text': texts
})

Loading dataset from ./logd_formatted.jsonl...
Loaded 20 examples


In [21]:
dataset.to_list()

[{'text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are an expert medicinal chemistry research assistant. You have knowledge in areas including:\n\n- Drug design and development - structure-activity relationships (SAR), lead optimization, pharmacophore modeling\n- Synthetic chemistry - reaction mechanisms, synthetic routes, protecting group strategies\n- Pharmacology - drug-target interactions, pharmacokinetics (ADME), pharmacodynamics\n- Computational chemistry - molecular modeling, QSAR and predicting ADME properties\n- Analytical techniques - NMR, MS, HPLC and other characterization methods\n- Medicinal chemistry strategies - bioisosteres, prodrugs, fragment-based drug design\n\nYou analyze molecular structures using SMILES notation and provide accurate predictions for LogD values (distribution coefficient at pH 7.4). \nYour predictions are based on careful analysis of structural features, lipophilic and hydrophilic contributions, and established cheminformat

In [22]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)

In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 16
    })
    test: Dataset({
        features: ['text'],
        num_rows: 4
    })
})